In [1]:
import requests
import obi_one as obi
virtual_lab_id=obi.LAB_ID_STAGING_TEST
project_id=obi.PROJECT_ID_STAGING_TEST
from entitysdk import Client, ProjectContext

from obi_auth import get_token
from obi_notebook import get_projects

obi_one_api_url = "http://127.0.0.1:8100"

token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=obi.LAB_ID_STAGING_TEST, project_id=obi.PROJECT_ID_STAGING_TEST)
db_client = Client(api_url="https://staging.openbraininstitute.org/api/entitycore", project_context=project_context, token_manager=token)

[2025-12-10 11:31:40,721] INFO: Local server listening on http://localhost:57149
[2025-12-10 11:31:40,721] INFO: Authentication url: https://staging.openbraininstitute.org/auth/realms/SBO/protocol/openid-connect/auth?response_type=code&client_id=obi-entitysdk-auth&redirect_uri=http%3A%2F%2Flocalhost%3A57149%2Fcallback&scope=openid&code_challenge=5MPDhTYaA4eONDydin9LJ4nBMOEvBxzj2BFw3bYOz7g&code_challenge_method=S256&kc_idp_hint=github
[2025-12-10 11:31:41,992] INFO: HTTP Request: POST https://staging.openbraininstitute.org/auth/realms/SBO/protocol/openid-connect/token "HTTP/1.1 200 OK"


# Imports

In [ ]:
# All essential imports needed for the notebook
from IPython.display import display, HTML
from obi_auth import get_token
from obi_notebook import get_projects, get_entities, get_environment
from entitysdk.models import EMCellMesh

## Authentication and project selection
We authenticate with the OBI platform.
### Project selection
Select from the dropdown menu the project that the output (a morphology with extracted spines) should be registered to. It will be available only in that project context. 

The widget lists all projects you have access to.

In [2]:
environment = get_environment.get_environment()
token = get_token(environment=environment, auth_mode="daf")
project_context = get_projects.get_projects(token)

NameError: name 'get_environment' is not defined

## Display EMCellMesh table

Input into the skeletonization process is a mesh describing the surface of a neuron. 

Below we will display a list of all surface meshes you have access to in the project context you selected above. This is a combination of publicly available entities open to all users and cell meshes you or your team have registered inside the project.
You can filter the list based on the EM dataset the meshes originate from. You can also search for specific neuron identifiers (typically called "pt_root_id").

Please select one mesh from the list below.

In [ ]:
mesh_ids = []
mesh_ids = get_entities.get_entities(
    'em-cell-mesh', token, [], env=environment, project_context=project_context, 
    multi_select=False, page_size=20, add_columns=["dense_reconstruction_cell_id"])

## Check the selection 

In [ ]:
if len(mesh_ids) == 0:
    display(HTML("<h3 style='color:#EDB95E'>⚠️ No mesh selected</h3>"))
    raise SystemExit()
mesh_id = mesh_ids[0]
input_entity = client.get_entity(entity_id=mesh_id, entity_type=EMCellMesh)

Prepare

In [ ]:
# Empty Simulation Configuration
sim_conf = obi.CircuitSimulationScanConfig.empty_config()

# Info
info = obi.Info(campaign_name="O1 Simulation", campaign_description="Simulation of O1 circuit with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Initialization

simulations_initialize = obi.CircuitSimulationScanConfig.Initialize(circuit=obi.CircuitFromID(id_str=circuit_id), 
                                                            node_set=sim_neuron_set.ref, 
                                                            simulation_length=sim_duration)

sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

# Generate GridScanGenerationTask (locally) - Note strange issue described for relative paths

In [ ]:
import tempfile

# Create a temporary directory (not used in this example, but can be useful for testing)
with tempfile.TemporaryDirectory() as temp_dir:

    """
    Very strange issue with adding assets when a relative path is used here. Need to look into after CNS.
    """
    # grid_scan = obi.GridScanGenerationTask(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='../../obi-output/circuit_simulations/grid_scan')
    grid_scan = obi.GridScanGenerationTask(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='/Users/james/Documents/obi/code/obi-output/circuit_simulations/grid_scan')
    # grid_scan = obi.GridScanGenerationTask(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root=temp_dir)
    grid_scan.multiple_value_parameters(display=True)
    grid_scan.coordinate_parameters(display=True)
    grid_scan.execute(db_client=db_client)
    obi.run_tasks_for_generated_scan(grid_scan, db_client=db_client)
